<a href="https://colab.research.google.com/github/StevenGustafson/portable_info_structures/blob/main/imt542_Sp2025_i3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# use pandas library and their dataframe data structure https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
# the pip command installs the library, which will work in Colab
# if using in your own machine, requires the pip package manager to be installed, or use another package manager
pip install pandas

In [2]:
# import the requests library, which is for making requests over HTTP to websites or APIs
# import the pandas library and rename it as pd for simplicity
import requests
import pandas as pd

In [31]:
# the tickers_cik json response will be
tickers_cik = pd.json_normalize(pd.json_normalize(tickers_cik.json(),\
max_level=0).values[0])

# the CIK code is used below to request other data about the company, so we will transform it into the correct format
# that is padded with 0 to 10 characters
tickers_cik["cik_str"] = tickers_cik["cik_str"].astype(str).str.zfill(10)

# a dataframe has an index, if you want to use an existing column, you need to set it
tickers_cik.set_index("ticker",inplace=True)

In [33]:
# Get the assets data for Apple, which has the CIK 0000320193 and the request requires 'CIK' to be added
response = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/Assets.json", headers=headers)

In [43]:
# transform the response into a flat structure using US Dollars as a Unit
assets_timeseries = pd.json_normalize(response.json()["units"]["USD"])

# convert the field 'end' in the dataframe to be a datetime, end stores the date aligned to asset amount
assets_timeseries["end"] = pd.to_datetime(assets_timeseries["end"])

# order the timeseries using the field that stores the date regarding the asset amount "end"
assets_timeseries = assets_timeseries.sort_values("end")

In [44]:
# import plotly library to draw a graph

import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(
    x=assets_timeseries["end"],
    y=assets_timeseries["val"],
    name='Total Assets value (USD)',
))
fig.update_layout(
    width=1000,
    height=700,
    paper_bgcolor='white',
    plot_bgcolor='#fafafa',
    hovermode='closest',
    title="Apple Total Assets value over time",
    xaxis = dict(
        title="Time"
    ),
    yaxis = dict(
        title="Total Assets value (USD)"
    ),
    showlegend=False)
fig.show()